### Setting

In [21]:
!pip install tqdm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/2b/78/57043611a16c655c8350b4c01b8d6abfb38cc2acb475238b62c2146186d7/tqdm-4.67.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 439.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer w

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from tqdm import tqdm


import pandas as pd
import numpy as np

import pickle
import time
import re

In [2]:
# pd.set_option('display.max_rows',None)

In [3]:
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9221")
chrome_driver = "/Applications/Google\ Chrome.app/Contents/MacOS/Google"

In [4]:
services = ChromeService(executable_path = chrome_driver)
driver = webdriver.Chrome(options=chrome_options)

### 각 회사 Review Summary 수집

In [56]:
corporate = pd.read_csv('sustainalytics_corporate.csv')
sustain_corpor_list = corporate['Company'].to_list()

In [58]:
company_review_df = pd.DataFrame(columns=['company', 'search_company', 'review'])

for company in sustain_corpor_list:
    print('회사명:', company)
    
    company_dict = {}
    company_dict['company'] = company
    
    # company url 접속
    time.sleep(2)
    url = 'https://www.glassdoor.com/Reviews/index.htm?overall_rating_low=3.5&page=1&locId=1147401&locType=C&occ=Data%20Sienctist'
    driver.get(url)
    time.sleep(5)
    
    # 검색창 탐색
    search_xpath = '//*[@id="companyAutocomplete-companyDiscover-employerSearch"]'
    search_element = driver.find_element(By.XPATH, search_xpath)  # 요소를 찾는 코드
    time.sleep(5)
    
    # send_keys로 값 입력
    search_element.send_keys(company)
    time.sleep(5)
    
    # 제출 버튼 클릭 
    button_xpath = driver.find_element(By.XPATH, '//*[@id="Explore"]/div[2]/div/div/div[2]/button')
    button_xpath.click()  # 버튼 클릭
    time.sleep(5)
    
    try:
        # 첫번째 검색 결과 link 출력
        search_list_xpath = '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]'
        search_list = driver.find_element(By.XPATH, search_list_xpath)
        time.sleep(3)
        first_result = search_list.find_elements(By.CSS_SELECTOR, 'a')[0]
        
        # 회사명
        company_nm_path = '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]/a[1]/div[2]/h3'
        company_nm_element = driver.find_element(By.XPATH, company_nm_path)
        time.sleep(3)
        company_nm = company_nm_element.text
        print('검색 회사명: ', company_nm)
        company_dict['search_company'] = company_nm

        # 리뷰 수
        review_path = '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]/a[1]/div[2]/div[4]/div/span[1]'
        review_element = driver.find_element(By.XPATH, review_path)
        time.sleep(3)
        num_of_review = review_element.text
        print('리뷰 수:', num_of_review)
        company_dict['review'] = num_of_review

        # df merge
        new_df = pd.DataFrame([company_dict])
        # 기존 DataFrame에 추가
        company_review_df = pd.concat([company_review_df, new_df], ignore_index=True)
        
        # 링크 이동
        # first_result_link = first_result.get_attribute('href')
        # driver.get(first_result_link)
        # time.sleep(7)

        # 리뷰 url 이동
        # a_elements = driver.find_element(By.CSS_SELECTOR, '#reviews > a')
        # time.sleep(5)
        # href = a_elements.get_attribute("href")
        # print(href)
        
    except:
        print('검색 결과가 없습니다.')
        company_dict['search_company'] = np.NaN
        company_dict['review'] = np.NaN

        new_df = pd.DataFrame([company_dict])
        company_review_df = pd.concat([company_review_df, new_df], ignore_index=True)

회사명: Adobe, Inc.
검색 회사명:  Adobe
리뷰 수: 11K
회사명: Advance Auto Parts, Inc.
검색 회사명:  Advance Auto Parts
리뷰 수: 5K
회사명: Advanced Micro Devices, Inc.
검색 회사명:  AMD
리뷰 수: 5K
회사명: AGCO Corp.
검색 회사명:  AGCO Corporation
리뷰 수: 7
회사명: Agilent Technologies, Inc.
검색 회사명:  Agilent Technologies
리뷰 수: 3K
회사명: Air Products & Chemicals, Inc.
검색 회사명:  Air Products
리뷰 수: 2K
회사명: Akamai Technologies, Inc.
검색 회사명:  Akamai
리뷰 수: 3K
회사명: Altus Group Ltd.
검색 회사명:  Altus Group
리뷰 수: 786
회사명: Amdocs Ltd.
검색 회사명:  Amdocs
리뷰 수: 13K
회사명: American Homes 4 Rent
검색 회사명:  AMH
리뷰 수: 635
회사명: American Tire Distributors, Inc.
검색 회사명:  American Tire Distributors
리뷰 수: 568
회사명: American Tower Corp.
검색 회사명:  American Tower
리뷰 수: 1K
회사명: Annaly Capital Management, Inc.
검색 회사명:  Annaly Capital Management
리뷰 수: 29
회사명: Applied Materials, Inc.
검색 회사명:  Applied Materials
리뷰 수: 5K
회사명: AptarGroup, Inc.
검색 회사명:  Aptar
리뷰 수: 593
회사명: Arrow Electronics, Inc.
검색 회사명:  Arrow Electronics
리뷰 수: 3K
회사명: Automatic Data Processing, Inc.
검색 회사명:

In [61]:
# 원본 저장
company_review_df.to_csv('company_summary.csv', index=False)

# 전처리 저장
company_review_df['review'] = company_review_df['review'].str.replace('K', '000')
company_review_df['review'] = company_review_df['review'].fillna(0).astype(int)

company_review_df.to_csv('company_review_summary.csv', index=False)

In [101]:
company_review_df[company_review_df['review']>500].reset_index(drop=True)
# company_review_df[company_review_df['review']>1000]

,company,search_company,review
0,"Adobe, Inc.",Adobe,11000
1,"Advance Auto Parts, Inc.",Advance Auto Parts,5000
2,"Advanced Micro Devices, Inc.",AMD,5000
3,"Agilent Technologies, Inc.",Agilent Technologies,3000
4,"Air Products & Chemicals, Inc.",Air Products,2000
5,"Akamai Technologies, Inc.",Akamai,3000
6,Altus Group Ltd.,Altus Group,786
7,Amdocs Ltd.,Amdocs,13000
8,American Homes 4 Rent,AMH,635
9,"American Tire Distributors, Inc.",American Tire Distributors,568
